This is an imperfect way to split the Charges in the Ministry of Defence's spreadsheet of court martial results (https://www.gov.uk/government/publications/court-martial-results-from-the-military-court-centres)

Notes:
- There are still some residual issues with the code, due to the sheer number of different ways the charges have been inputted, as well as some issues with handling desirable punctual; e.g. commas mid sentence as part of a list vs undesirably punctual such as & . / - \ etc etc
- However these charges have to manually coded after being outputted so there is an opportunity for human oversight to fix the remaining issues

In [3]:
import pandas as pd

data = pd.read_csv('Original_Data_2010-23.csv')

df = pd.DataFrame(data)

df

,Reference number,Rank,Service,Unit,Trial Court,Sentencing Date,Year of Sentencing,Charge (s),Act charged under,Finding,Guilty > 0
0,NaN,Signaller,Army,NaN,Colchester,07-Jan-10,2010,"2 x Battery, 1 x Assault Occasioning Actual Bo...",NaN,"1 x Not Guilty, 2 x Guilty",Yes
1,NaN,Colonel,Army,NaN,Sennelager,08-Jan-10,2010,"5 x Obtaining a money transfer by deception, 6...",NaN,"11 x Not Guilty, 2 x Guilty",Yes
2,NaN,Guardsman,Army,NaN,Colchester,13-Jan-10,2010,1 x Absence Without Leave,NaN,Guilty,Yes
3,NaN,Signaller,Army,NaN,Colchester,13-Jan-10,2010,1 x Absence Without Leave,NaN,Guilty,Yes
4,NaN,Guardsman,Army,NaN,Colchester,21-Jan-10,2010,"1 x Desertion, 1 x Absence Without Leave",NaN,"Not Guilty, Guilty",Yes
...,...,...,...,...,...,...,...,...,...,...,...
6066,6106,Lance Corporal,Army,212 Yorkshire Field Hospital,Catterick,14-Dec-23,2023,Ch 1: Sexual assault Ch 2-4: Disgraceful condu...,NaN,Ch 1-3: Not Guilty\n Ch 4 & 5: Guilty\n Ch 6: ...,Yes
6067,6245,Ex-Chief Petty Officer,Royal Navy,Formerly of HMS Drake,Bulford,14-Dec-23,2023,Ch 1a: Sexual assault\n Ch 1b: Alternative cha...,NaN,"Ch 1a, 2a, 5a: Not Guilty - no evidence offere...",Yes
6068,6317,Sergeant,Army,26 Regt RA,Bulford,15-Dec-23,2023,Ch 1: Threatening with an offensive weapon.,NaN,Guilty,Yes
6069,6352,Flight Lieutenant,Royal Air Force,RAF Brize Norton,Bulford,15-Dec-23,2023,Ch 1 & 2: Contravention of standing orders,NaN,Guilty,Yes


In [31]:
import pandas as pd
import re

def clean_charge_descriptions(charges):
    cleaned_charges = []

    # Define regex patterns to identify where to split based on common charge indicators
    split_pattern = r'(?=\b\d+\s*x\b|\bCh\b\.?\s*\d+|\bCharge[s]?\b)'

    # Regex to remove unwanted elements like "Ch", "Charge", and numbers
    patterns_to_remove = [
        (r'\bCh\b\.?\s*\d*[:]?(\s*-?\s*\d+)?\s*', ''),
        (r'\bCharge[s]?\b', ''),
        (r'\b\d+\s*x\b', ''),
        (r'\b\d+\b', '')
    ]

    # Remove all unwanted punctuation except commas, and then remove trailing commas
    punctuation_to_remove = r'[^\w\s,]'

    for charge in charges:
        clean_charge = charge if pd.notna(charge) else ""

        # Split based on the identified charge patterns
        split_entries = re.split(split_pattern, clean_charge)
        split_entries = [entry.strip() for entry in split_entries if entry.strip()]

        for entry in split_entries:
            for pattern, repl in patterns_to_remove:
                entry = re.sub(pattern, repl, entry)
            entry = re.sub(punctuation_to_remove, '', entry)
            entry = re.sub(r',\s*$', '', entry)
            if entry:
                cleaned_charges.append(entry.strip())

    return cleaned_charges


# Apply cleaning to the Charge (s) column and convert results into a new DataFrame
cleaned_charge_list = clean_charge_descriptions(data['Charge (s)'])
cleaned_charge_df = pd.DataFrame(cleaned_charge_list, columns=['Cleaned Charge'])

# Display a sample of the cleaned data
cleaned_charge_df.head(20)

,Cleaned Charge
0,Battery
1,Assault Occasioning Actual Bodily Harm
2,Obtaining a money transfer by deception
3,Fraud
4,Forgery
5,Making a false entry in an official document
6,Absence Without Leave
7,Absence Without Leave
8,Desertion
9,Absence Without Leave


In [36]:
print(len(cleaned_charge_df))

8249


In [34]:
unique_cleaned_charge_df = cleaned_charge_df.drop_duplicates().reset_index(drop=True)

unique_cleaned_charge_df.head(20)

,Cleaned Charge
0,Battery
1,Assault Occasioning Actual Bodily Harm
2,Obtaining a money transfer by deception
3,Fraud
4,Forgery
5,Making a false entry in an official document
6,Absence Without Leave
7,Desertion
8,Attempted sexual assault
9,Drunkenness


In [35]:
print(len(unique_cleaned_charge_df))

1348


In [33]:
cleaned_csv_path = 'Cleaned_Charge_Data.csv'

# Export the cleaned data to a CSV file
unique_cleaned_charge_df.to_csv(cleaned_csv_path, index=False)

cleaned_csv_path

'Cleaned_Charge_Data.csv'